In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, accuracy_score
from sklearn.impute import SimpleImputer
import joblib  # For saving/loading model
import os      # For file existence check

In [2]:

# --- 1. Load dataset ---
try:
    df = pd.read_excel("new.xlsx")
except FileNotFoundError:
    print("Error: The file 'new.xlsx' was not found.")
    exit()

# Drop unnecessary columns
if "Unnamed: 0" in df.columns:
    df = df.drop(columns=["Unnamed: 0"])

# Copy original dataframe
df_original = df.copy()

In [3]:
# --- 2. Handle categorical variables ---
cat_cols = df.select_dtypes(include=['object']).columns
print("Categorical Columns:", list(cat_cols))

label_encoders = {}
for col in cat_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col].astype(str))
    label_encoders[col] = le

Categorical Columns: ['appointment_id', 'customer_id', 'booking_date', 'booking_time', 'booking_datetime', 'created_at', 'service_type', 'staff_assigned', 'status', 'weather', 'channel', 'name', 'email', 'gender', 'city', 'joined_date', 'tags', 'primary_business', 'primary_service']


In [4]:
# --- 3. Split Features & Target ---
# Assuming 'status' is the target column
X = df.drop(columns=["status"])
y = df["status"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Handle missing values
imputer = SimpleImputer(strategy="constant", fill_value=0)
X_train = imputer.fit_transform(X_train)
X_test = imputer.transform(X_test)

In [5]:
# --- 4. Train Gradient Boosting Classifier ---
print("\n--- Model Training: Gradient Boosting Classifier ---")
model = GradientBoostingClassifier(
    n_estimators=100,
    learning_rate=0.0092,
    subsample=0.84555,
    max_depth=10,
    random_state=42
)
model.fit(X_train, y_train)
print("✅ Gradient Boosting Classifier trained.")



--- Model Training: Gradient Boosting Classifier ---
✅ Gradient Boosting Classifier trained.


In [6]:
# --- 5. Evaluate ---
y_pred = model.predict(X_test)

print("\nAccuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))


Accuracy: 0.6825

Classification Report:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00       300
           1       0.67      1.00      0.80      1571
           2       0.00      0.00      0.00       189
           3       0.00      0.00      0.00       273
           4       1.00      1.00      1.00        67

    accuracy                           0.68      2400
   macro avg       0.33      0.40      0.36      2400
weighted avg       0.47      0.68      0.55      2400



C:\Users\Hp\anaconda3\envs\ankitaenv\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Hp\anaconda3\envs\ankitaenv\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Hp\anaconda3\envs\ankitaenv\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [7]:
# --- 6. Save the trained model and LabelEncoders ---
print("\n--- Saving the trained model and LabelEncoders ---")
joblib.dump(model, "mango_model.pkl")
joblib.dump(label_encoders, "mango_label_encoders.pkl")
print("✅ Model and LabelEncoders saved successfully as 'mango_model.pkl' and 'mango_label_encoders.pkl'.")


--- Saving the trained model and LabelEncoders ---
✅ Model and LabelEncoders saved successfully as 'mango_model.pkl' and 'mango_label_encoders.pkl'.


In [8]:
# --- 7. Load the saved model and LabelEncoders ---
print("\n--- Loading the saved model and LabelEncoders ---")
if os.path.exists("mango_model.pkl") and os.path.exists("mango_label_encoders.pkl"):
    loaded_model = joblib.load("mango_model.pkl")
    loaded_label_encoders = joblib.load("mango_label_encoders.pkl")
    print("✅ Model and LabelEncoders loaded successfully.")
else:
    print("Error: Saved model or encoder files not found.")
    exit()



--- Loading the saved model and LabelEncoders ---
✅ Model and LabelEncoders loaded successfully.


In [9]:
# --- 8. Test with new user input ---
print("\n--- Testing with User Input ---")
user_input_data = {}

for col in X.columns:
    if col in cat_cols:
        le = loaded_label_encoders[col]
        options = le.classes_
        print(f"\nSelect a value for '{col}':")
        for i, option in enumerate(options):
            print(f"[{i}] {option}")
        while True:
            try:
                choice_index = int(input("Enter the number corresponding to your choice: "))
                if 0 <= choice_index < len(options):
                    value = options[choice_index]
                    break
                else:
                    print("Invalid choice. Please enter a valid number.")
            except ValueError:
                print("Invalid input. Please enter a number.")
    else:
        value = input(f"Enter value for '{col}': ")
    user_input_data[col] = [value]

# Convert input to DataFrame
user_df = pd.DataFrame(user_input_data)

# Encode categorical columns
for col in cat_cols:
    if col in user_df.columns:
        le = loaded_label_encoders[col]
        user_df[col] = le.transform(user_df[col])

# Ensure numeric format
user_df = user_df.apply(pd.to_numeric)

# Prediction
prediction = loaded_model.predict(user_df)

print("\nPrediction for your input:")
print(f"The predicted status is: {prediction[0]}")



--- Testing with User Input ---

Select a value for 'appointment_id':
[0] APPT0000001
[1] APPT0000002
[2] APPT0000003
[3] APPT0000004
[4] APPT0000005
[5] APPT0000006
[6] APPT0000007
[7] APPT0000008
[8] APPT0000009
[9] APPT0000010
[10] APPT0000011
[11] APPT0000012
[12] APPT0000013
[13] APPT0000014
[14] APPT0000015
[15] APPT0000016
[16] APPT0000017
[17] APPT0000018
[18] APPT0000019
[19] APPT0000020
[20] APPT0000021
[21] APPT0000022
[22] APPT0000023
[23] APPT0000024
[24] APPT0000025
[25] APPT0000026
[26] APPT0000027
[27] APPT0000028
[28] APPT0000029
[29] APPT0000030
[30] APPT0000031
[31] APPT0000032
[32] APPT0000033
[33] APPT0000034
[34] APPT0000035
[35] APPT0000036
[36] APPT0000037
[37] APPT0000038
[38] APPT0000039
[39] APPT0000040
[40] APPT0000041
[41] APPT0000042
[42] APPT0000043
[43] APPT0000044
[44] APPT0000045
[45] APPT0000046
[46] APPT0000047
[47] APPT0000048
[48] APPT0000049
[49] APPT0000050
[50] APPT0000051
[51] APPT0000052
[52] APPT0000053
[53] APPT0000054
[54] APPT0000055
[55]

Enter the number corresponding to your choice:  1199



Select a value for 'customer_id':
[0] CUST000001
[1] CUST000002
[2] CUST000003
[3] CUST000004
[4] CUST000005
[5] CUST000006
[6] CUST000007
[7] CUST000008
[8] CUST000009
[9] CUST000010
[10] CUST000011
[11] CUST000012
[12] CUST000013
[13] CUST000014
[14] CUST000015
[15] CUST000016
[16] CUST000017
[17] CUST000018
[18] CUST000019
[19] CUST000020
[20] CUST000021
[21] CUST000022
[22] CUST000023
[23] CUST000024
[24] CUST000025
[25] CUST000026
[26] CUST000027
[27] CUST000028
[28] CUST000029
[29] CUST000030
[30] CUST000031
[31] CUST000032
[32] CUST000033
[33] CUST000034
[34] CUST000035
[35] CUST000036
[36] CUST000037
[37] CUST000038
[38] CUST000039
[39] CUST000040
[40] CUST000041
[41] CUST000042
[42] CUST000043
[43] CUST000044
[44] CUST000045
[45] CUST000046
[46] CUST000047
[47] CUST000048
[48] CUST000049
[49] CUST000050
[50] CUST000051
[51] CUST000052
[52] CUST000053
[53] CUST000054
[54] CUST000055
[55] CUST000056
[56] CUST000057
[57] CUST000058
[58] CUST000059
[59] CUST000060
[60] CUST000061

Enter the number corresponding to your choice:  299



Select a value for 'booking_date':
[0] 2023-09-07
[1] 2023-09-08
[2] 2023-09-09
[3] 2023-09-10
[4] 2023-09-11
[5] 2023-09-12
[6] 2023-09-13
[7] 2023-09-14
[8] 2023-09-15
[9] 2023-09-16
[10] 2023-09-17
[11] 2023-09-18
[12] 2023-09-19
[13] 2023-09-20
[14] 2023-09-21
[15] 2023-09-22
[16] 2023-09-23
[17] 2023-09-24
[18] 2023-09-25
[19] 2023-09-26
[20] 2023-09-27
[21] 2023-09-28
[22] 2023-09-29
[23] 2023-09-30
[24] 2023-10-01
[25] 2023-10-02
[26] 2023-10-03
[27] 2023-10-04
[28] 2023-10-05
[29] 2023-10-06
[30] 2023-10-07
[31] 2023-10-08
[32] 2023-10-09
[33] 2023-10-10
[34] 2023-10-11
[35] 2023-10-12
[36] 2023-10-13
[37] 2023-10-14
[38] 2023-10-15
[39] 2023-10-16
[40] 2023-10-17
[41] 2023-10-18
[42] 2023-10-19
[43] 2023-10-20
[44] 2023-10-21
[45] 2023-10-22
[46] 2023-10-23
[47] 2023-10-24
[48] 2023-10-25
[49] 2023-10-26
[50] 2023-10-27
[51] 2023-10-28
[52] 2023-10-29
[53] 2023-10-30
[54] 2023-10-31
[55] 2023-11-01
[56] 2023-11-02
[57] 2023-11-03
[58] 2023-11-04
[59] 2023-11-05
[60] 2023-11-0

...

Enter the number corresponding to your choice:  1



Select a value for 'primary_service':
[0] AC Repair
[1] Bike Service
[2] Birthday Booking
[3] Car Service
[4] Career
[5] Catering
[6] Cleaning
[7] Consultation
[8] Dine-in
[9] Engine Repair
[10] Extraction
[11] Facial
[12] Filling
[13] Fridge Repair
[14] Hair Color
[15] Haircut
[16] Legal
[17] Manicure
[18] Microwave Repair
[19] Nutrition
[20] Orthodontics
[21] Pedicure
[22] Reservation
[23] Tax
[24] Tyre Change
[25] Washing Machine Repair


Enter the number corresponding to your choice:  5



Prediction for your input:
The predicted status is: 4


C:\Users\Hp\anaconda3\envs\ankitaenv\lib\site-packages\sklearn\utils\validation.py:2732: UserWarning: X has feature names, but GradientBoostingClassifier was fitted without feature names
  warnings.warn(


# import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, accuracy_score
from sklearn.impute import SimpleImputer
import joblib  # For saving/loading model
import os      # For file existence check

# --- 1. Load dataset ---
try:
    df = pd.read_excel("new.xlsx")
except FileNotFoundError:
    print("Error: The file 'new.xlsx' was not found.")
    exit()

# Drop unnecessary columns
if "Unnamed: 0" in df.columns:
    df = df.drop(columns=["Unnamed: 0"])

# Copy original dataframe
df_original = df.copy()

# --- 2. Handle categorical variables ---
cat_cols = df.select_dtypes(include=['object']).columns
print("Categorical Columns:", list(cat_cols))

label_encoders = {}
for col in cat_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col].astype(str))
    label_encoders[col] = le

# --- 3. Split Features & Target ---
# Assuming 'status' is the target column
X = df.drop(columns=["status"])
y = df["status"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Handle missing values
imputer = SimpleImputer(strategy="constant", fill_value=0)
X_train = imputer.fit_transform(X_train)
X_test = imputer.transform(X_test)

# --- 4. Train Gradient Boosting Classifier ---
print("\n--- Model Training: Gradient Boosting Classifier ---")
model = GradientBoostingClassifier(
    n_estimators=500,
    learning_rate=0.0092,
    subsample=0.84555,
    max_depth=10,
    random_state=42
)
model.fit(X_train, y_train)
print("✅ Gradient Boosting Classifier trained.")

# --- 5. Evaluate ---
y_pred = model.predict(X_test)

print("\nAccuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

# --- 6. Save the trained model and LabelEncoders ---
print("\n--- Saving the trained model and LabelEncoders ---")
joblib.dump(model, "mango_model.pkl")
joblib.dump(label_encoders, "mango_label_encoders.pkl")
print("✅ Model and LabelEncoders saved successfully as 'mango_model.pkl' and 'mango_label_encoders.pkl'.")

# --- 7. Load the saved model and LabelEncoders ---
print("\n--- Loading the saved model and LabelEncoders ---")
if os.path.exists("mango_model.pkl") and os.path.exists("mango_label_encoders.pkl"):
    loaded_model = joblib.load("mango_model.pkl")
    loaded_label_encoders = joblib.load("mango_label_encoders.pkl")
    print("✅ Model and LabelEncoders loaded successfully.")
else:
    print("Error: Saved model or encoder files not found.")
    exit()

# --- 8. Test with new user input ---
print("\n--- Testing with User Input ---")
user_input_data = {}

for col in X.columns:
    if col in cat_cols:
        le = loaded_label_encoders[col]
        options = le.classes_
        print(f"\nSelect a value for '{col}':")
        for i, option in enumerate(options):
            print(f"[{i}] {option}")
        while True:
            try:
                choice_index = int(input("Enter the number corresponding to your choice: "))
                if 0 <= choice_index < len(options):
                    value = options[choice_index]
                    break
                else:
                    print("Invalid choice. Please enter a valid number.")
            except ValueError:
                print("Invalid input. Please enter a number.")
    else:
        value = input(f"Enter value for '{col}': ")
    user_input_data[col] = [value]

# Convert input to DataFrame
user_df = pd.DataFrame(user_input_data)

# Encode categorical columns
for col in cat_cols:
    if col in user_df.columns:
        le = loaded_label_encoders[col]
        user_df[col] = le.transform(user_df[col])

# Ensure numeric format
user_df = user_df.apply(pd.to_numeric)

# Prediction
prediction = loaded_model.predict(user_df)

print("\nPrediction for your input:")
print(f"The predicted status is: {prediction[0]}")


In [ ]:
# --- 7. Load the saved model and LabelEncoders ---
print("\n--- Loading the saved model and LabelEncoders ---")
if os.path.exists("mango_model.pkl") and os.path.exists("mango_label_encoders.pkl"):
    loaded_model = joblib.load("mango_model.pkl")
    loaded_label_encoders = joblib.load("mango_label_encoders.pkl")
    print("✅ Model and LabelEncoders loaded successfully.")
else:
    print("Error: Saved model or encoder files not found.")
    exit()

# --- 8. Test with new user input ---
print("\n--- Testing with User Input ---")
user_input_data = {}

for col in X.columns:
    if col in cat_cols:
        le = loaded_label_encoders[col]
        options = le.classes_
        print(f"\nSelect a value for '{col}':")
        for i, option in enumerate(options):
            print(f"[{i}] {option}")
        while True:
            try:
                choice_index = int(input("Enter the number corresponding to your choice: "))
                if 0 <= choice_index < len(options):
                    value = options[choice_index]
                    break
                else:
                    print("Invalid choice. Please enter a valid number.")
            except ValueError:
                print("Invalid input. Please enter a number.")
    else:
        value = input(f"Enter value for '{col}': ")
    user_input_data[col] = [value]

# Convert input to DataFrame
user_df = pd.DataFrame(user_input_data)

# Encode categorical columns
for col in cat_cols:
    if col in user_df.columns:
        le = loaded_label_encoders[col]
        user_df[col] = le.transform(user_df[col])

# Ensure numeric format
user_df = user_df.apply(pd.to_numeric)

# Prediction
prediction = loaded_model.predict(user_df)

print("\nPrediction for your input:")
print(f"The predicted status is: {prediction[0]}")
